In [61]:
from nbt_structure_utils import NBTStructure, Vector, Cuboid, BlockData
from nbt_structure_utils.nbt_structure import EMPTY_SPACE
from nbt.nbt import TAG_Compound, TAG_Byte, TAG_List, TAG_String
from nbt import nbt
import os
import json
from itertools import product
import re
import unicodedata
import copy

Sto notebook serve a fixare cose delle rovine conduit che non avevo voglia di fixare ingame

(overengineering? Probabile, ma fight me)

In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
folder = "../../Mod Fabric/src/main/resources/data/growsseth/structures/ruins/conduit_ruins/"
as_new_files = False

In [64]:
# per stare sicuri che ci stiano nei cartelli più fitti
ignore_newlines = False
file_signs = {
    "main": [
        "Trono del mare",
    ],
    "maze": [
        "Tre anelli intersecati corpo, anima e spirito a dio vincolati",
        "Di pietra marina\nsua pelle e vestigia",
        "E in petto il suo cuore sopito\nattende il contatto del mare",
        "Cosi' infine il nostro legame il nostro condotto e' ristabilito.",
    ],
    "maze_end": [
        "Erede del nostro culto\nprendi il cuore nel suo guscio",
        "risali e sull'altare\nconcludi il tuo percorso",
    ]
}

In [65]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
def format(string: str):
    processed = string.lower().strip()
    processed = re.sub(r'(?<=\w)\'(?=\w)', ' ', processed)
    processed = strip_accents(processed)
    processed = re.sub(r'[^\w\s]+', '', processed)
    return processed


In [66]:
# MAX_SIGN_LINE_WIDTH = 90
APPROX_MAX_LINE_CHARS = 17

_file_signs = copy.deepcopy(file_signs)

# Formato minecraft
def to_raw_json(string):
    return json.dumps({
        'text': string,
        'bold': False,
        'font': 'minecraft:alt',
    })

# grazie chatgpt, non avevo voglia
def split_words_into_strings(words, max_length):
    result_strings = []
    current_string = ""
    
    for word in words:
        if len(current_string) + len(word) + 1 <= max_length:
            if current_string:
                current_string += " "
            current_string += word
        else:
            result_strings.append(current_string)
            current_string = word
    
    if current_string:
        result_strings.append(current_string)
    
    return result_strings

def make_sign_data(message, color="white", waxed=False, glowing_text=False):
    message = format(message)
    line_pieces = message.split('\n')
    if max(map(len, line_pieces)) > APPROX_MAX_LINE_CHARS:
        line_pieces = []
        for line in message.split('\n'):
            line_pieces.extend(split_words_into_strings(line.split(" "), APPROX_MAX_LINE_CHARS))

    from_line = 0
    if len(line_pieces) <= 2:
        from_line = 1

    is_waxed_tag = TAG_Byte(1 if waxed else 0, name="is_waxed")
    is_glowing_tag = TAG_Byte(1 if glowing_text else 0, name="has_glowing_text")
    color_tag = TAG_String(color, name="color")

    messages_tag = TAG_List(name="messages", type=TAG_String)
    messages_tag.tags.extend([TAG_String(to_raw_json("")) for _ in range(4 - len(line_pieces))])
    for piece in line_pieces[::-1]:
        messages_tag.tags.insert(from_line, TAG_String(to_raw_json(piece)))
    messages_tag.tags = messages_tag.tags[:4]

    front_text = TAG_Compound(name="front_text")
    front_text.tags.extend([
        messages_tag,
        color_tag,
        is_glowing_tag,
    ])

    # back_text = TAG_Compound(name="back_text")
    # back_text.tags.extend([
    #     messages_tag,
    #     color_tag,
    #     is_glowing_tag,
    # ])

    # common tags
    id_tag = TAG_String(name="id", value="Sign")
    
    nbt_data = TAG_Compound()
    nbt_data.tags.extend([
        # sign tags
        is_waxed_tag, 
        front_text, 
        # back_text,
        # common tags
        id_tag,
    ])

    return nbt_data

for file in _file_signs:
    print(f"\n==== {file} ====\n")
    path = os.path.join(folder, file + ".nbt")
    sign_lines = file_signs[file]
    nbtstructure = NBTStructure(path)
    if as_new_files:
        out_file = path.replace(".nbt", "_edit.nbt")
    else:
        out_file = path

    for idx in nbtstructure.blocks:
        block_pos = nbtstructure.blocks[idx]
        pos = block_pos.pos
        block = nbtstructure.palette[block_pos.state]
        if block and block.name == "minecraft:warped_wall_sign":
            line = sign_lines.pop()
            if ignore_newlines:
                line = line.replace('\n', ' ')
            nbt_data = make_sign_data(line, "white", True, True)
            nbtstructure.set_block(pos, block, None, nbt_data)
            print(nbtstructure.blocks.get(pos).other_nbt.pretty_tree())
            if len(sign_lines) == 0:
                break
        
    nbtstructure.get_nbt(pressurize=False).write_file(filename=out_file)


==== main ====

TAG_Compound: {3 Entries}
{
	TAG_Byte('is_waxed'): 1
	TAG_Compound('front_text'): {3 Entries}
	{
		TAG_List('messages'): [4 TAG_String(s)]
		{
			TAG_String: {"text": "", "bold": false, "font": "minecraft:alt"}
			TAG_String: {"text": "trono del mare", "bold": false, "font": "minecraft:alt"}
			TAG_String: {"text": "", "bold": false, "font": "minecraft:alt"}
			TAG_String: {"text": "", "bold": false, "font": "minecraft:alt"}
		}
		TAG_String('color'): white
		TAG_Byte('has_glowing_text'): 1
	}
	TAG_String('id'): Sign
}

==== maze ====

TAG_Compound: {3 Entries}
{
	TAG_Byte('is_waxed'): 1
	TAG_Compound('front_text'): {3 Entries}
	{
		TAG_List('messages'): [4 TAG_String(s)]
		{
			TAG_String: {"text": "cosi infine il", "bold": false, "font": "minecraft:alt"}
			TAG_String: {"text": "nostro legame", "bold": false, "font": "minecraft:alt"}
			TAG_String: {"text": "il nostro", "bold": false, "font": "minecraft:alt"}
			TAG_String: {"text": "condotto", "bold": false, "font":